In [ ]:
!pip install -q aicrowd-cli
%load_ext aicrowd.magic
API_KEY = '' #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 76 kB 3.3 MB/s 
     |████████████████████████████████| 208 kB 36.1 MB/s 
     |████████████████████████████████| 62 kB 1.0 MB/s 
     |████████████████████████████████| 170 kB 44.6 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
API Key valid
Saved API Key successfully!


In [ ]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c docking-iss -o data

submission.csv:   0%|          | 0.00/267k [00:00<?, ?B/s]

test.zip:   0%|          | 0.00/75.1M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

train.zip:   0%|          | 0.00/150M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

val.zip:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

In [ ]:
!unzip data/train.zip -d data/train > /dev/null
!unzip data/val.zip -d data/val > /dev/null
!unzip data/test.zip -d data/test > /dev/null

!pip install -q timm
!pip install -qU albumentations
!pip install -q efficientnet_pytorch

     |████████████████████████████████| 376 kB 7.2 MB/s 
     |████████████████████████████████| 98 kB 5.1 MB/s 
     |████████████████████████████████| 37.1 MB 1.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import glob
from tqdm.notebook import tqdm
import cv2
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.cuda import amp

from sklearn.metrics import cohen_kappa_score
from efficientnet_pytorch import EfficientNet
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import datasets, models, transforms
import timm
import os
import time
import math
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import KFold

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
import albumentations as albu

from sklearn.metrics import accuracy_score, roc_auc_score
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import math
import os
import random
import shutil
import time
import warnings
from collections import Counter, defaultdict
from contextlib import contextmanager
from functools import partial
from pathlib import Path
from google.colab.patches import cv2_imshow

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
OUTPUT_DIR = './'

class CFG:
    epochs = 50
    batch_size = 4
    target_col = 'sign'
    model_name = 'tf_efficientnetv2_l'
    size = 512
    pretrained = True
    target_size = 3
    pin_memory = False
    seed = 777
    target_col = 'sign'
    coeff = 15

    num_workers = 2
    gradient_accumulation_steps = 1
    apex = False
    max_grad_norm = 1000
    print_freq = 10
    factor = 0.1
    patience = 3
    eps = 1e-6
    T_max = 10
    T_0 = 10
    min_lr = 1e-6
    lr = 1e-4
    weight_decay = 1e-6
    t1 = 0.3
    t2 = 1.0
    smoothing = 0.05

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df, directory, transform=None):
        self.df = df
        self.directory = directory
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df['ImageID'][idx]
        file_path = f'{self.directory}/{img_id}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        distance = self.df[self.df['ImageID'] == img_id]['distance'].values[0]
        location = eval(self.df[self.df["ImageID"]==img_id]['location'].values[0])

        label = [distance/CFG.coeff, location[0]/CFG.coeff, location[1]/CFG.coeff]
        return image, label

In [ ]:
def get_transforms(data=''):
    if data == 'train':
        return albu.Compose([
                             albu.Resize(CFG.size, CFG.size),
                             albu.Blur(blur_limit=7, p=0.4),
                             albu.OpticalDistortion(p=0.1),
                             albu.GridDistortion(p=0.1),
                             albu.IAAPiecewiseAffine(p=0.1),
                             albu.HueSaturationValue(hue_shift_limit=7, sat_shift_limit=5, val_shift_limit=7, p=0.3),
                             albu.RandomBrightnessContrast(p=0.2),
                             albu.RandomGamma(p=0.2),
                             ToTensorV2()
        ])
    elif data == 'valid':
        return albu.Compose([
                             albu.Resize(CFG.size, CFG.size),
                             ToTensorV2()
        ])

In [ ]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f"[{name}] start")
    yield
    LOGGER.info(f"[{name}] done in {time.time() - t0:.0f} s.")


def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data_time.update(time.time() - end)
        images = images.to(device)

        y = [i.to(torch.float32).to(device).unsqueeze(1) for i in y]

        batch_size = images.size(0)

        pred = model(images.float())

        loss = 2 * criterion(pred[0], y[0]) + 2 * criterion(pred[1], y[1]) + criterion(pred[2], y[2])

        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, y) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        # measure data loading time
        data_time.update(time.time() - end)
        batch_size = images.size(0)
        images = images.to(device)
        #labels = labels.view(-1, 1)
        y = [i.to(torch.float32).to(device).unsqueeze(1) for i in y]
        # compute loss
        with torch.no_grad():
            pred = model(images.float())

        loss = criterion(pred[0], y[0]) + criterion(pred[1], y[1])
        losses.update(loss.item(), batch_size)
        # record accuracy
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
    return losses.avg

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(CFG.model_name, pretrained=CFG.pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential()
        self.distance_fc = nn.Linear(self.n_features, 1)
        self.location_fc_x = nn.Linear(self.n_features, 1)
        self.location_fc_y = nn.Linear(self.n_features, 1)

    def forward(self, x):
        batch_size = x.shape[0]
        output = self.model(x)
        
        distance = self.distance_fc(output)
        location_x = self.location_fc_x(output)
        location_y = self.location_fc_y(output)
        return distance, location_x, location_y

In [ ]:
def train_loop():
    train_df = pd.read_csv('data/train.csv')
    valid_df = pd.read_csv('data/val.csv')

    train_dataset = TrainDataset(train_df, 'data/train', transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_df, 'data/val', transform=get_transforms(data='valid'))

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers,
        pin_memory=CFG.pin_memory,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=CFG.pin_memory,
        drop_last=False,
    )
    
    model = CustomModel()
    model.to(device)

    if device == torch.device("cuda"):
        model = torch.nn.DataParallel(model)

    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=1)

    criterion = nn.MSELoss()

    best_score = 0.0
    best_loss = np.inf

    for epoch in range(CFG.epochs):
        start_time = time.time()

        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        avg_val_loss = valid_fn(valid_loader, model, criterion, device)
        scheduler.step(avg_loss)

        elapsed = time.time() - start_time

        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} time: {elapsed:.0f}s"
        )
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f"Epoch {epoch+1} - Save Best loss: {best_loss:.6f} Model")
            torch.save(model.state_dict(), f"model.pth")

In [ ]:
train_loop()

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:290: FutureWarning: This IAAPiecewiseAffine is deprecated. Please use PiecewiseAffine instead
  warnings.warn("This IAAPiecewiseAffine is deprecated. Please use PiecewiseAffine instead", FutureWarning)
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_l-d664b728.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_l-d664b728.pth


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 - avg_train_loss: 194.0004  avg_val_loss: 3.0848 time: 1706s


INFO     Epoch 1 - avg_train_loss: 194.0004  avg_val_loss: 3.0848 time: 1706s

...

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 32 - avg_train_loss: 0.5958  avg_val_loss: 0.0455 time: 1712s


INFO     Epoch 32 - avg_train_loss: 0.5958  avg_val_loss: 0.0455 time: 1712s

  0%|          | 0/2500 [00:00<?, ?it/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-12-6590b82f66d8>:1 in <module>                                             │
│ <ipython-input-11-d0dd195402a2>:42 in train_loop                                          │
│ <ipython-input-9-8038f85928e1>:99 in train_fn                                             │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/torch/_tensor.py:255 in backward                   │
│                                                                                           │
│    252 │   │   │   │   retain_graph=retain_graph,                                         │
│    253 │   │   │   │   create_graph=create_graph,                                         │
│    254 │   │   │   │   inputs=inputs)                                                     │
│ ❱  255 │   │   torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs │
│    256 │                                                                                  │
│    257 │   def register_hook(self, hook):                                                 │
│    258 │   │   r"""Registers a backward hook.                                             │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:149 in backward         │
│                                                                                           │
│   146 │                                                                                   │
│   147 │   Variable._execution_engine.run_backward(                                        │
│   148 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                 │
│ ❱ 149 │   │   allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag     │
│   150                                                                                     │
│   151                                                                                     │
│   152 def grad(                                                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
class InferDataset(Dataset):
    def __init__(self, df, directory, transform=None):
        self.df = df
        self.directory = directory
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df['ImageID'][idx]
        file_path = f'{self.directory}/{img_id}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, img_id

In [ ]:
model = CustomModel()
model.load_state_dict(torch.load('model.pth'))
model.eval()
model.to(device)

test_df = pd.read_csv('data/submission.csv')
test_dataset = InferDataset(test_df, 'data/test', transform=get_transforms(data='valid'))
test_loader = DataLoader(
    test_dataset,
    batch_size = 2,
    shuffle=False,
    pin_memory=CFG.pin_memory,
    num_workers=CFG.num_workers
)
distances, locations, imageids = [], [], []
for images, img_ids in tqdm(test_loader):
    images = images.to(device)
    pred = model(images.float())

    distance = np.concatenate(pred[0].cpu().detach().tolist())
    distances.extend([*distance])

    location_x = pred[1].cpu().detach().tolist()
    location_y = pred[2].cpu().detach().tolist()
    location = [[x[0] * CFG.coeff, y[0] * CFG.coeff]  for x, y in zip(location_x, location_y)]
    locations.extend(location)

    imageids.extend([*img_ids.tolist()])
distances = np.array(distances) * CFG.coeff

  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
test_dataset.df.ImageID = imageids
test_dataset.df.distance = distances
test_dataset.df.location = locations

In [ ]:
test_dataset.df

,ImageID,distance,location
0,0,335.675926,"[262.07244873046875, 202.57806301116943]"
1,1,474.480858,"[319.1917133331299, 95.70791244506836]"
2,2,450.982590,"[223.8416576385498, 467.47899055480957]"
3,3,125.815601,"[288.8049602508545, 481.3499450683594]"
4,4,154.912920,"[231.83680057525635, 433.29219818115234]"
...,...,...,...
4995,4995,448.383751,"[227.97399044036865, 205.25911331176758]"
4996,4996,350.900602,"[258.8722515106201, 497.4689483642578]"
4997,4997,378.514881,"[231.9409418106079, 478.3934497833252]"
4998,4998,66.541607,"[317.33656883239746, 350.2694034576416]"


In [ ]:
!rm -rf assets
!mkdir assets

test_dataset.df.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [ ]:
!aicrowd notebook submit -c docking-iss -a assets